In [14]:
years = list(range(1994, 2022))

In [15]:
url_start = "https://www.pro-football-reference.com/draft/{}-combine.htm"

In [16]:
import requests

for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open("combine/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [17]:
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
dfs = []
for year in years:
    with open("combine/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_='over_header').decompose()
    draft_table = soup.find(id='combine')
    combines = pd.read_html(str(draft_table))[0]
    combines['Year'] = year
    
    dfs.append(combines)

AttributeError: 'NoneType' object has no attribute 'decompose'